# GAM Coach: Binary Classification

In this notebook, we will learn how to generate *diverse* and *customizable* counterfactual explanations for Generalized Additive Models (GAMs).

## 2. Train an EBM Model

Let's train a binary classification model using EBM on the Lending Club dataset. The dataset contains financial information (18 features) of 5000 loan applicants. The target variable has a binary value: `1` if the person paid off the loan and `0` if the person failed to pay off the loan. To learn more about how we generate this dataset, check out our [GAM Coach paper]().

We can use this dataset to train a binary classifier to make future loan decisions (only for demo purpose).

In [28]:
%load_ext autoreload
%autoreload 2

import importlib
import gamcoach as coach

import numpy as np
import pandas as pd

import urllib.request
import json
import pickle

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter

SEED = 101221

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Load a pre-processed lending club dataset
ca_data_url = 'https://gist.githubusercontent.com/xiaohk/06266553d43e591817914bfe52ec9b60/raw/c190b7cea837739797336d690fe44df9d8f9384c/lending-club-data-5000-ca.json'

with urllib.request.urlopen(ca_data_url) as url:
    data = json.loads(url.read().decode())

# Load the training data
x_all = np.array(data['x_all'])
y_all = np.array(data['y_all']) # `y_all`: 1 if paid off, 0 if failed to pay off

# Load some meta data
feature_names = data['feature_names']
feature_types = data['feature_types']
cont_index = data['cont_index']
cat_index = data['cat_index']

# Use float to encode continuous features and string for categorical features
for i, t in enumerate(feature_types):
    if t == 'continuous':
        x_all[:, i] = x_all[:, i].astype(float)
    elif t == 'categorical':
        x_all[:, i] = x_all[:, i].astype(str)

In [19]:
# Create a dataFrame to validate that we load the correct data
df = pd.DataFrame(x_all)
df.columns = feature_names

df.head()

,loan_amnt,term,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,fico_score
0,10000.0,36 months,3 years,RENT,4.662757831681574,Source Verified,debt_consolidation,15.11,1998.0,8.0,1,3.693990610460777,23.0,12.0,Individual,0,1,707.0
1,12000.0,36 months,3 years,RENT,4.653212513775344,Verified,debt_consolidation,26.29,1999.0,14.0,0,4.35173834558076,64.4,27.0,Individual,0,0,692.0
2,9600.0,36 months,8 years,RENT,4.653212513775344,Verified,debt_consolidation,20.05,2000.0,22.0,1,3.6242820958356683,17.4,35.0,Individual,0,1,687.0
3,35000.0,60 months,10+ years,MORTGAGE,5.1303337684950066,Verified,debt_consolidation,17.29,1991.0,15.0,2+,4.161787118745523,74.4,21.0,Individual,1,0,677.0
4,14500.0,36 months,6 years,RENT,4.623249290397901,Not Verified,credit_card,17.89,2005.0,10.0,0,3.843855422623161,59.7,19.0,Individual,0,0,707.0


In [32]:
# Train a binary classifier
x_train, x_test, y_train, y_test = train_test_split(x_all,
                                                    y_all,
                                                    test_size=0.2,
                                                    random_state=SEED)

# Use sample weight to combat class imbalance
weight = np.bincount(y_train)[1] / np.bincount(y_train)[0]
x_train_sample_weights = [weight if y_train[i] == 0 else 1 for i in range(len(y_train))]

# ebm = ExplainableBoostingClassifier(feature_names,
#                                     feature_types,
#                                     random_state=SEED)

# ebm.fit(x_train, y_train, sample_weight=x_train_sample_weights)
# pickle.dump(ebm, open('./model_pickles/lending-binary.pkl', 'wb'))

ebm = pickle.load(open('./model_pickles/lending-binary.pkl', 'rb'))

In [33]:
# Evaluate our model

y_pred = ebm.predict(x_test)
y_pred_prob = ebm.predict_proba(x_test)[:, 1]

print(Counter(y_pred))
print()

accuracy = metrics.accuracy_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred_prob)
f1 = metrics.f1_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

tn = confusion_matrix[0, 0]
fn = confusion_matrix[1, 0]
fp = confusion_matrix[0, 1]
tp = confusion_matrix[1, 1]
specificity = tn / (tn + fp)

temp = ('accuracy: {:.4f} \nauc: {:.4f} \nrecall:{:.4f} \nprecision: {:.4f} '+
    'specificity: {:.4f} \nf1: {:.4f}\nbalanced accuracy: {:.4f}')
print(temp.format(accuracy, auc, recall, precision, specificity, f1, balanced_accuracy))
print()

print('confusion matrix:\n', confusion_matrix)

Counter({1: 622, 0: 378})

accuracy: 0.6500 
auc: 0.6729 
recall:0.6722 
precision: 0.8537 specificity: 0.5667 
f1: 0.7521
balanced accuracy: 0.6194

confusion matrix:
 [[119  91]
 [259 531]]


## 3. Generate Counterfactual Explanations

In [34]:
# Find an interesting data point
# We can focus on test cases where our model rejcets the application (y_hat = 0)
reject_index = y_pred == 0
x_reject = x_test[reject_index, :]
y_pred_reject = y_pred[reject_index]

reject_df = pd.DataFrame(np.hstack((x_reject, y_pred_reject.reshape(-1, 1))))
reject_df.columns = feature_names + ['prediction']
print(reject_df.shape)
reject_df.head()

(378, 19)


,loan_amnt,term,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,fico_score,prediction
0,10000.0,36 months,< 1 year,OWN,4.531478917042255,Source Verified,credit_card,31.66,2008.0,7.0,0,3.760422483423212,51.0,12.0,Individual,0,0,677.0,0
1,10000.0,60 months,1 year,RENT,4.643452676486188,Not Verified,debt_consolidation,4.34,2014.0,7.0,0,3.671358003443492,26.7,7.0,Individual,0,0,662.0,0
2,11000.0,36 months,8 years,MORTGAGE,4.556302500767287,Source Verified,major_purchase,13.4,2011.0,12.0,1,3.8899736384039962,44.9,15.0,Individual,0,1,677.0,0
3,15400.0,60 months,3 years,RENT,4.763427993562937,Verified,debt_consolidation,27.18,2005.0,9.0,0,3.940217555599735,68.1,28.0,Individual,0,0,682.0,0
4,10725.0,60 months,4 years,RENT,4.698970004336019,Source Verified,house,27.63,2004.0,9.0,0,3.688864568054792,20.5,19.0,Individual,0,0,772.0,0


In [35]:
# Find a random sample
rs = np.random.RandomState(SEED)
target_index = rs.choice(range(reject_df.shape[0]))
cur_example = x_reject[target_index, :]

print(cur_example)

['16750.0' '60 months' 'missing' 'RENT' '4.510866590733636' 'Verified'
 'debt_consolidation' '16.65' '2001.0' '4.0' '0' '3.867113779831977'
 '75.9' '9.0' 'Individual' '0' '0' '702.0']


In [38]:
# Generate counterfactual (CF) explanations
my_coach = coach.GAMCoach(ebm, x_train)

In [25]:
# To generate good CF explantions, we need to consider what features are mutable
# and they possible values

cfs = my_coach.generate_cfs(
    cur_example,
    total_cfs=3,
    # List of features that the CFs can change
    features_to_vary=['loan_amnt', 'term', 'emp_length', 'home_ownership',
                      'annual_inc', 'purpose', 'dti', 'open_acc', 'revol_bal',
                      'revol_util', 'total_acc', 'application_type', 'mort_acc',
                      'fico_score'],
    # Some continuous features need to have integer values in practice
    continuous_integer_features=['open_acc', 'total_acc', 'mort_acc', 'fico_score']
)

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


ValueError: invalid literal for int() with base 10: '158,30'

In [ ]:
# View counterfactual exampels as a dataframe
cfs.to_df()

,loan_amnt,term,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,fico_score,new_prediction
0,16750.0,36 months,missing,RENT,4.510866590733636,Verified,debt_consolidation,16.65,2001.0,4.0,0,3.867113779831977,75.9,8.0,Individual,0,0,702.0,1
1,16750.0,60 months,missing,RENT,4.510866590733636,Verified,debt_consolidation,16.65,2001.0,4.0,0,3.577334315474175,75.9,7.0,Individual,0,0,702.0,0
2,16987.5,60 months,missing,RENT,4.634476101197249,Verified,debt_consolidation,19.255000000000003,2001.0,4.0,0,3.867113779831977,75.9,5.0,Individual,0,0,715.0,0


In [ ]:
# View counterfactual examples as "strategies" in text format
cfs.show()

## Strategy 1 ##
Change <term> from "60 months" to "36 months" 
	* score gain: 0.7060
	* distance cost: 0.5554
Change <total_acc> from 9.0 to 8.0 [7.5, 8.5)
	* score gain: 0.2304
	* distance cost: 0.1429

## Strategy 2 ##
Change <revol_bal> from 3.867113779831977 to 3.577334315474175 [3.568201581311275, 3.577434315474175)
	* score gain: 0.7290
	* distance cost: 1.1480
Change <total_acc> from 9.0 to 7.0 [6.5, 7.5)
	* score gain: 0.2204
	* distance cost: 0.2857

## Strategy 3 ##
Change <loan_amnt> from 16750.0 to 16987.5 [16987.5, 17025.0)
	* score gain: 0.0132
	* distance cost: 0.0396
Change <annual_inc> from 4.510866590733636 to 4.634476101197249 [4.634476101197249, 4.642066414881926)
	* score gain: 0.2979
	* distance cost: 0.8048
Change <dti> from 16.65 to 19.255000000000003 [19.255000000000003, 19.325)
	* score gain: 0.1215
	* distance cost: 0.4480
Change <total_acc> from 9.0 to 5.0 [4.5, 5.5)
	* score gain: 0.2031
	* distance cost: 0.5714
Change <fico_score> from 702.0 to 715.0 [714